BART


In [1]:
import pandas as pd
import joblib
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the dataset (pre-classified with 'primary_region' column)
dataset_path = "processed_testdataset.csv"
data = pd.read_csv(dataset_path)

# OPTIONAL: Load your classification model if reclassification is needed
classifier = joblib.load("logistic_regression_model.pkl")


In [2]:
# Load the BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


In [ ]:
# Function to generate a summary for a single article
def generate_summary(article_text):
    # Tokenize and encode the input
    inputs = tokenizer.encode("summarize: " + article_text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate the summary
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the output
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Apply summarization to each article in the dataset
data['summary'] = data['article'].apply(generate_summary)

# Save the summarized dataset
summarized_dataset_path = "path_to_save_summarized_dataset.csv"
data.to_csv(summarized_dataset_path, index=False)

print(f"Summarized dataset saved to {summarized_dataset_path}")


In [5]:
# Install required libraries
!pip install transformers pandas rouge-score nltk
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

T5

In [ ]:


# Import libraries
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

# Load T5 model and tokenizer
model_name = "t5-small"  # Use 't5-base' or 't5-large' for larger models
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to summarize text using T5
def t5_summarize(article, max_length=150, min_length=40):
    inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Load the dataset into a pandas DataFrame
df = pd.read_csv(dataset_path)  # Adjust to read_excel or read_json as needed
# Summarize each article in the dataset
# Ensure the column name 'article' matches your dataset's column for text data
df['generated_summary'] = df['article'].apply(lambda x: t5_summarize(x))

# Define functions to calculate ROUGE and BLEU scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def calculate_rouge(reference, generated):
    scores = scorer.score(reference, generated)
    return scores['rouge1'].fmeasure, scores['rouge2'].fmeasure, scores['rougeL'].fmeasure

def calculate_bleu(reference, generated):
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    return sentence_bleu([reference_tokens], generated_tokens)

# Compute ROUGE and BLEU scores
rouge1_scores, rouge2_scores, rougeL_scores, bleu_scores = [], [], [], []

for index, row in df.iterrows():
    if pd.notna(row['highlights']):  # Ensure the reference column (e.g., 'highlights') exists and is not empty
        rouge1, rouge2, rougeL = calculate_rouge(row['highlights'], row['generated_summary'])
        rouge1_scores.append(rouge1)
        rouge2_scores.append(rouge2)
        rougeL_scores.append(rougeL)
        bleu_scores.append(calculate_bleu(row['highlights'], row['generated_summary']))

# Add scores to the DataFrame
df['rouge1'] = rouge1_scores
df['rouge2'] = rouge2_scores
df['rougeL'] = rougeL_scores
df['bleu'] = bleu_scores

# Calculate and print average scores
avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores) if rouge2_scores else 0
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0
avg_bleu = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0

print(f"Average ROUGE-1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2: {avg_rouge2:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")
print(f"Average BLEU: {avg_bleu:.4f}")

# Save the dataset with scores to a CSV file
output_file = "summarized_dataset_with_scores_t5.csv"
df.to_csv(output_file, index=False)

print("Summarization complete! Summarized dataset saved as:", output_file)


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 13.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24970 sha256=8b004fd09713403bd5812c1b578eb3df3e80586c17082ccf5dd6446d6ffbee38
  Stored in directory: c:\users\hasan\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


c:\Users\hasan\AppData\Local\Programs\Python\Python312\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\hasan\AppData\Local\Programs\Python\Python312\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\hasan\AppData\Local\Programs\Python\Python312\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lowe

Average ROUGE-1: 0.3613
Average ROUGE-2: 0.1547
Average ROUGE-L: 0.2535
Average BLEU: 0.0535
Summarization complete! Summarized dataset saved as: summarized_dataset_with_scores_t5.csv


In [6]:

# Import libraries
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load the Pegasus model and tokenizer
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)


df = pd.read_csv(dataset_path)  # Change to read_excel or read_json as needed
# Define a function to summarize text
def summarize_text(text, max_length=60, min_length=10):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Generate summaries
df['generated_summary'] = df['article'].apply(lambda x: summarize_text(x))

# Define functions to calculate ROUGE and BLEU scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def calculate_rouge(reference, generated):
    scores = scorer.score(reference, generated)
    return scores['rouge1'].fmeasure, scores['rouge2'].fmeasure, scores['rougeL'].fmeasure

def calculate_bleu(reference, generated):
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    return sentence_bleu([reference_tokens], generated_tokens)

# Compute scores for each row
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
bleu_scores = []

for index, row in df.iterrows():
    if pd.notna(row['highlights']):  # Ensure reference summaries are available
        rouge1, rouge2, rougeL = calculate_rouge(row['highlights'], row['generated_summary'])
        rouge1_scores.append(rouge1)
        rouge2_scores.append(rouge2)
        rougeL_scores.append(rougeL)
        bleu_scores.append(calculate_bleu(row['highlights'], row['generated_summary']))

# Add scores to the DataFrame
df['rouge1'] = rouge1_scores
df['rouge2'] = rouge2_scores
df['rougeL'] = rougeL_scores
df['bleu'] = bleu_scores

# Calculate and print average scores
avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)
avg_bleu = sum(bleu_scores) / len(bleu_scores)

print(f"Average ROUGE-1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2: {avg_rouge2:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")
print(f"Average BLEU: {avg_bleu:.4f}")

# Save the dataset with scores to a CSV file
output_file = "summarized_dataset_with_scores_pegasus.csv"
df.to_csv(output_file, index=False)

# Download the dataset
print("Summarization complete! Summarized dataset saved as:", output_file)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\hasan\AppData\Local\Programs\Python\Python312\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\hasan\AppData\Local\Programs\Python\Python312\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider u

Average ROUGE-1: 0.2996
Average ROUGE-2: 0.1089
Average ROUGE-L: 0.2047
Average BLEU: 0.0217
Summarization complete! Summarized dataset saved as: summarized_dataset_with_scores_pegasus.csv
